# Dataset Loading

In [ ]:
# Data loading and processing 
import pandas as pd 
import numpy as np
import json
whole_df = pd.read_csv("C:/Users/Sharafat/Desktop/Masters' Dissertation/Structured_whole_dataframe.csv")
whole_df = whole_df.drop("Unnamed: 0", axis = 1)
train_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Train/")]
train_df
train_df["type"] = train_df["type"].replace("No interaction", "no interaction")
train_df
test_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Test/")]
test_df
test_df["type"] = test_df["type"].replace("No interaction", "no interaction")
test_df
# example => train_df or test_df 
train_df_records = train_df.to_dict(orient = "records")
train_df_records
test_df_records = test_df.to_dict(orient = "records")
test_df_records
# 1. Sample 10 examples of 'no interaction'
no_interaction_df = train_df[train_df['type'] == 'no interaction'].sample(n=10, random_state=42)

# 2. Sample 3 examples from each of the *other* types (mechanism, effect, etc.)
other_types_df = (
    train_df[train_df['type'] != 'no interaction']
    .groupby('type', group_keys=False)
    .sample(n=3, random_state=42)
)

# 3. Combine and shuffle
examples_df = pd.concat([no_interaction_df, other_types_df]).sample(frac=1, random_state=42).reset_index(drop=True)

examples_df["type"] = examples_df["type"].replace("No interaction", "no interaction")
examples_df
example_shots = examples_df.to_dict(orient="records")
example_shots
train_df_grouped = train_df.groupby("sentence_id")
train_df_grouped
group = train_df_grouped.get_group("DDI-DrugBank.d64.s87")
group
train_df["triplet"] = list(zip(train_df["e1_text"], train_df["type"], train_df["e2_text"]))
train_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("train_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)

with open("train_df_sentence_triplets.json","r") as f:
    train_triplets = json.load(f)

train_triplets[0]
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("test_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)
with open("test_df_sentence_triplets.json","r") as f:
    test_triplets = json.load(f)

test_triplets[0]
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))
    
    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("train_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("train_df_sentence_entities.json","r") as f:
    train_entities = json.load(f)

train_entities[0]
train_triplets
few_shot_examples = train_triplets
few_shot_examples
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))
    
    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("test_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("test_df_sentence_entities.json","r") as f:
    test_entities = json.load(f)

test_entities[0]
len(train_entities)
len(test_entities)
examples_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = examples_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("few_shot_example_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)

with open("few_shot_example_triplets.json","r") as f:
    few_shot_entities = json.load(f)

few_shot_entities[0]
len(few_shot_entities)
train_entities
train_triplets # ground truth, remove interaction_type and choose random examples for few_shot
def transform_structure(data):
    output = {'sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

transformed = transform_structure(train_entities[0])
print(transformed)

def transform_structure(data):
    output = {'Sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

updated_train_entities = []
for i in range(len(train_entities)):
    transformed = transform_structure(train_entities[i])
    updated_train_entities.append(transformed)
print(updated_train_entities[-1])
len(updated_train_entities)

# GPT v2 improved model

In [ ]:
with open("random_few_shot_examples_2.json","r") as f:
    few_shot_prompt_2_examples = json.load(f)

In [ ]:
with open("random_few_shot_examples_5.json","r") as f:
    few_shot_prompt_5_examples = json.load(f)

In [ ]:
with open("random_few_shot_examples_10.json","r") as f:
    few_shot_prompt_10_examples = json.load(f)

In [ ]:
with open("random_few_shot_examples_20.json","r") as f:
    few_shot_prompt_20_examples = json.load(f)

In [ ]:
import os
import json
import time
from google import genai


GEMINI_API_KEY = ''
assert GEMINI_API_KEY, ""
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
client = genai.Client()

# Output holder
all_responses = []

# Prompt styles
interaction_types = """
- mechanism: Pharmacokinetic interaction (metabolism, clearance, absorption, transport). Cues: inhibits/induces, substrate, CYP/UGT/P-gp, AUC/Cmax change, plasma levels ↑/↓, bioavailability, clearance.
- effect: Pharmacodynamic clinical effect or outcome. Cues: bleeding, sedation, QT prolongation, hypotension, serotonin syndrome, reduced efficacy, toxicity, adverse events.
- advice: Explicit recommendation/warning/contraindication. Cues: avoid, do not use, contraindicated, use with caution, monitor, reduce/increase dose, separate by X hours.
- int: A generic mention that an interaction exists without mechanism, effect, or advice. Cues: “interacts”, “interaction exists”, “DDI reported”, “potential interaction” with no further details.
- no interaction: Explicit statement of no interaction or safe coadministration (e.g., “no significant interaction”, “can be coadministered”, “safe with”).
"""

rules = """ Rules (strict):
A. Allowed labels (lowercase) are exactly: mechanism, effect, advice, int, no interaction.
   - Never output "advise". If your reasoning suggests "advise", output the label "advice".
B. One triplet per unique unordered pair of entities. Do not output both (Drug1, x, Drug2) and (Drug2, x, Drug1).
C. Decision hierarchy (to avoid misuse of "int"):
   1) If any recommendation/warning/contraindication language appears → advice.
   2) Else if PK level/process is stated (inhibit/induce/CYP/levels/AUC/clearance/absorption/transport) → mechanism.
   3) Else if clinical effect/outcome/AE/risk is stated (bleeding, sedation, etc.) → effect.
   4) Else if the text only states there is an interaction with no details → int.
   5) Else → no interaction.
D. “Monitor”, “use with caution”, “dose adjust”, “separate dosing” are **advice**, not int.
E. “Levels/concentrations/AUC/Cmax change”, “CYP/UGT/P-gp”, “inhibits/induces/substrate” are **mechanism**, not int.
F. “Increased risk of [event]” or any clinical outcome is **effect**, not int.
G. Output only triplets in the form: (Drug1, label, Drug2). No extra text. """

few_shot_rules = """ Rules (strict):
A. Allowed labels (lowercase) are exactly: mechanism, effect, advice, int, no interaction.
   - Never output "advise". If your reasoning suggests "advise", output the label "advice".
B. One triplet per unique unordered pair of entities. Do not output both (Drug1, x, Drug2) and (Drug2, x, Drug1).
C. Decision hierarchy (to avoid misuse of "int"):
   1) advice > 2) mechanism > 3) effect > 4) int > 5) no interaction.
D. “Monitor”, “use with caution”, “dose adjust”, “separate dosing” → advice.
E. PK cues (CYP/UGT/P-gp, levels/AUC/Cmax, inhibit/induce/substrate) → mechanism.
F. Clinical outcome/risk terms → effect.
G. Output only triplets in the exact form: (Drug1, label, Drug2). """

few_shot_examples = f"""
{interaction_types}
"""

def extract_entities(entity_dict):
    return [v for k, v in entity_dict.items() if k.startswith("Entity_")]

# Inference loop
for i in range(0, len(updated_train_entities)):
    example = updated_train_entities[i]
    sentence = example["Sentence"]
    entities = extract_entities(example)

    output_entry = {
        "index": i,
        "sentence": sentence,
        "entities": entities
    }

    # Prepare prompts
    zero_shot_prompt = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from the following text.

Definitions of interaction types: {interaction_types}
Rules are given: {rules}

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""

    few_shot_prompt_2 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

 Definitions of interaction types: {interaction_types}
 Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_2_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""
    
    few_shot_prompt_5 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types:{interaction_types}
Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_5_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""


    few_shot_prompt_10 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types: {interaction_types}
Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_10_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""
    
    few_shot_prompt_20 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types: {interaction_types}
Rules are given: {few_shot_rules}


Examples: {few_shot_prompt_20_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""


    cot_prompt = f"""You are a biomedical expert. Perform the following:

1) List unique unordered entity pairs.
2) For each pair, locate evidence span(s) in the sentence.
3) Apply the decision hierarchy strictly (advice > mechanism > effect > int > no interaction) considering definition of labels: {interaction_types}.
4) Canonicalize the label set to: mechanism, effect, advice, int, no interaction.
   - If you would output "advise", output "advice" instead.
5) Produce exactly one triplet per pair.

Rules are given: {rules}

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:
"""

    prompts = {
        "zero_shot_output": zero_shot_prompt,
        "few_shot_output_2": few_shot_prompt_2,
        "few_shot_output_5": few_shot_prompt_5,
        "few_shot_output_10": few_shot_prompt_10,
        "few_shot_output_20": few_shot_prompt_20,
        "cot_output": cot_prompt
    }

    for method_name, prompt in prompts.items():
        try:
            response = client.models.generate_content(
                model="gemma-3-27b-it",
                contents=prompt,
                config=genai.types.GenerateContentConfig(
                    temperature=0.0
                )
            )
            time.sleep(6)

            output_entry[method_name] = response.text.strip()
            print(f"[{method_name}] Finished index {i}")
            print(f"{method_name}: {response.text.strip()}")

        except Exception as e:
            print(f"Error at index {i} with method {method_name}: {e}")
            output_entry[method_name] = "ERROR"

    all_responses.append(output_entry)

    # Save after each example (optional safety)
    with open("GPT_Prompting_method_2.json", "w") as f:
        json.dump(all_responses, f, indent=2)

# Evaluation

In [ ]:
import json 

with open("GPT_Prompting_method_2_last.json", "r") as f:
    all_responses = json.load(f)
all_responses

In [ ]:
train_triplets_df = pd.DataFrame(train_triplets)
train_triplets_df
triplet_prediction_results = pd.DataFrame(all_responses)
triplet_prediction_results

In [ ]:
merged_df = train_triplets_df.merge(triplet_prediction_results)
merged_df

In [ ]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Configuration
INTERACTION_TYPES = {"effect", "mechanism", "advice", "int", "no interaction"}
MODEL_MAP = {
    'zero_shot_output': 'Zero-Shot',
    'few_shot_output_2': 'Few-Shot-2',
    'few_shot_output_5': 'Few-Shot-5',
    'few_shot_output_10': 'Few-Shot-10',
    'few_shot_output_20': 'Few-Shot-20',
    'cot_output': 'CoT'
}
MODEL_ORDER = list(MODEL_MAP.keys())

# --- Normalization ---
def normalize_interaction_type(itype: str) -> str:
    """Map variations of interaction types to canonical form"""
    itype = itype.strip().lower()
    if itype in {"advise", "adviced", "advised", "advice"}:
        return "advice"
    return itype

def normalize_drug_pair(drug1, drug2):
    """Create canonical drug pair representation"""
    return tuple(sorted([drug1.strip().lower(), drug2.strip().lower()]))

# --- Parsing ---
def parse_prediction_output(pred_output):
    """Robust parser that requires exactly 2 drugs and 1 interaction type"""
    if not isinstance(pred_output, str):
        return []
    
    triplet_pattern = r'\(([^,]+?),\s*([^,]+?),\s*([^)]+?)\)'
    raw_triplets = re.findall(triplet_pattern, pred_output)
    
    cleaned_triplets = []
    for triplet in raw_triplets:
        parts = [p.strip().lower() for p in triplet]
        norm_parts = [normalize_interaction_type(p) for p in parts]
        
        # Identify interaction type candidates
        interaction_candidates = [p for p in norm_parts if p in INTERACTION_TYPES]
        drug_candidates = [parts[i] for i, p in enumerate(norm_parts) if p not in INTERACTION_TYPES]
        
        if len(interaction_candidates) == 1 and len(drug_candidates) == 2:
            itype = interaction_candidates[0]
            cleaned_triplets.append((drug_candidates[0], itype, drug_candidates[1]))
            continue
        
        # Fallback: positional checks
        if len(parts) == 3:
            if norm_parts[1] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], norm_parts[1], parts[2]))
            elif norm_parts[0] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[1], norm_parts[0], parts[2]))
            elif norm_parts[2] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], norm_parts[2], parts[1]))
    
    return cleaned_triplets


# --- Evaluation ---
def evaluate_predictions(df):
    """Comprehensive evaluation with model comparison metrics"""
    results = []
    per_class_results = []
    
    for model_col in MODEL_ORDER:
        if model_col not in df.columns:
            continue
            
        model_name = MODEL_MAP[model_col]
        
        # Initialize metrics
        metrics = {
            'Model': model_name,
            'Accuracy': 0,
            'Micro_Precision': 0, 'Micro_Recall': 0, 'Micro_F1': 0,
            'Macro_Precision': 0, 'Macro_Recall': 0, 'Macro_F1': 0,
            'Weighted_Precision': 0, 'Weighted_Recall': 0, 'Weighted_F1': 0,
        }
        
        class_metrics = {itype: {'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0} for itype in INTERACTION_TYPES}
        all_true = []
        all_pred = []
        
        for _, row in df.iterrows():
            # Process ground truth
            gold_triplets = set()
            for triplet in row['triplets']:
                if len(triplet) == 3:
                    d1, itype, d2 = [x.strip().lower() for x in triplet]
                    itype = normalize_interaction_type(itype)
                    gold_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Process predictions
            pred_triplets = set()
            for d1, itype, d2 in parse_prediction_output(row[model_col]):
                itype = normalize_interaction_type(itype)
                pred_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Create lists for sklearn metrics
            for itype in INTERACTION_TYPES:
                # True positive: both gold and prediction have this interaction
                if any((dp, t) in gold_triplets and t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['TP'] += 1
                    all_true.append(itype)
                    all_pred.append(itype)
                # False positive: prediction has this interaction but gold doesn't
                elif any(t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['FP'] += 1
                    all_pred.append(itype)
                    # Assign the true label
                    true_label = "no interaction"
                    for dp, t in gold_triplets:
                        if dp in [dp_p for dp_p, t_p in pred_triplets if t_p == itype]:
                            true_label = t
                            break
                    all_true.append(true_label)
                # False negative: gold has this interaction but prediction doesn't
                elif any(t == itype for dp, t in gold_triplets):
                    class_metrics[itype]['FN'] += 1
                    all_true.append(itype)
                    all_pred.append("no interaction")  # Missed prediction
        
        # Calculate accuracy
        correct = sum(1 for t, p in zip(all_true, all_pred) if t == p)
        total = len(all_true) if all_true else 1
        metrics['Accuracy'] = correct / total if total > 0 else 0
        
        # Calculate micro, macro, and weighted metrics using sklearn
        if all_true and all_pred:
            micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='micro', zero_division=0)
            macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='macro', zero_division=0)
            weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='weighted', zero_division=0)
            
            metrics.update({
                'Micro_Precision': micro_precision,
                'Micro_Recall': micro_recall,
                'Micro_F1': micro_f1,
                'Macro_Precision': macro_precision,
                'Macro_Recall': macro_recall,
                'Macro_F1': macro_f1,
                'Weighted_Precision': weighted_precision,
                'Weighted_Recall': weighted_recall,
                'Weighted_F1': weighted_f1
            })
        
        results.append(metrics)
        
        # Calculate per-class metrics
        for itype in INTERACTION_TYPES:
            cm = class_metrics[itype]
            p = cm['TP'] / (cm['TP'] + cm['FP']) if cm['TP'] + cm['FP'] > 0 else 0
            r = cm['TP'] / (cm['TP'] + cm['FN']) if cm['TP'] + cm['FN'] > 0 else 0
            f = 2 * p * r / (p + r) if p + r > 0 else 0
            support = cm['TP'] + cm['FN']
            
            per_class_results.append({
                'Model': model_name,
                'Interaction': itype,
                'Precision': p,
                'Recall': r,
                'F1': f,
                'Support': support
            })
    
    return pd.DataFrame(results), pd.DataFrame(per_class_results)

# --- Report ---
def generate_comparison_report(global_df, class_df):
    """Create model comparison report with sorting"""
    global_df = global_df.sort_values('Micro_F1', ascending=False)
    
    # Format numbers
    global_df = global_df.round(4)
    class_df = class_df.round(4)
    
    report = "MODEL COMPARISON REPORT\n"
    report += "=======================\n\n"
    report += "Global Metrics:\n"
    report += global_df.to_string(index=False) + "\n\n"
    report += "Per-Class Metrics:\n"
    report += class_df.to_string(index=False)
    
    best_model = global_df.iloc[0]
    report += f"\n\nBest Model: {best_model['Model']} (Macro F1={best_model['Macro_F1']:.4f}, Micro F1={best_model['Micro_F1']:.4f})"
    
    return report

# --- Main execution ---
if __name__ == "__main__":
    global_results, class_results = evaluate_predictions(merged_df)
    report = generate_comparison_report(global_results, class_results)
    print(report)


In [ ]:
def calculate_pair_and_interaction_counts(merged_df):
    TECHNIQUE_ORDER = [
        'zero_shot_output',
        'few_shot_output_2',
        'few_shot_output_5',
        'few_shot_output_10',
        'few_shot_output_20',
        'cot_output'
    ]

    TECHNIQUE_MAP = {
        'zero_shot_output': 'Zero-Shot',
        'few_shot_output_2': 'Few-Shot (2)',
        'few_shot_output_5': 'Few-Shot (5)',
        'few_shot_output_10': 'Few-Shot (10)',
        'few_shot_output_20': 'Few-Shot (20)',
        'cot_output': 'Chain-of-Thought'
    }
    
    per_method_details = []
    per_interaction_details = []
    
    for tech_col in TECHNIQUE_ORDER:
        if tech_col not in merged_df.columns:
            continue
        
        tech_name = TECHNIQUE_MAP.get(tech_col, tech_col)
        
        interaction_correct, interaction_total = 0, 0
        per_class = {itype: {'tp': 0, 'fn': 0} for itype in INTERACTION_TYPES}
        
        for _, row in merged_df.iterrows():
            # Ground truth triplets
            gold_triplets = set()
            if isinstance(row['triplets'], list):
                for triplet in row['triplets']:
                    if len(triplet) == 3:
                        d1, itype, d2 = [x.strip().lower() for x in triplet]
                        itype = normalize_interaction_type(itype)  # <-- FIX
                        gold_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Predictions
            pred_triplets = set()
            for d1, itype, d2 in parse_prediction_output(row[tech_col]):
                itype = normalize_interaction_type(itype)  # <-- FIX
                pred_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Pair-level counts (correct if both pair+interaction match)
            gold_pairs = {dp for dp, _ in gold_triplets}
            for dp, it in pred_triplets:
                if dp in gold_pairs:  # model predicted some interaction for a true pair
                    interaction_total += 1
                    if (dp, it) in gold_triplets:
                        interaction_correct += 1
            
            # Per-interaction type counts
            for itype in INTERACTION_TYPES:
                gold_sub = {(dp, it) for dp, it in gold_triplets if it == itype}
                pred_sub = {(dp, it) for dp, it in pred_triplets if it == itype}
                
                tp = len(gold_sub & pred_sub)
                fn = len(gold_sub - pred_sub)
                
                per_class[itype]['tp'] += tp
                per_class[itype]['fn'] += fn
        
        # Store per-method totals
        per_method_details.append({
            "Technique": tech_name,
            "Correct Pairs": interaction_correct,
            "Total Pairs": interaction_total
        })
        
        # Store per-interaction type totals
        for itype in INTERACTION_TYPES:
            tp = per_class[itype]['tp']
            fn = per_class[itype]['fn']
            per_interaction_details.append({
                "Technique": tech_name,
                "Interaction Type": itype,
                "Correct": tp,
                "Total": tp + fn
            })
    
    return pd.DataFrame(per_method_details), pd.DataFrame(per_interaction_details)
method_df, interaction_df = calculate_pair_and_interaction_counts(merged_df)
print("PER-METHOD PAIR COUNTS:")
print(method_df.to_string(index=False))
print("\nPER-INTERACTION TYPE COUNTS:")
print(interaction_df.to_string(index=False))


In [ ]:
import matplotlib.pyplot as plt

def plot_class_distributions(class_df):
    """
    Visualize correct prediction distribution per class across models.
    Each bar shows recall (correct / total).
    """
    classes = sorted(class_df['Interaction'].unique())
    models = class_df['Model'].unique()

    # Make one subplot per interaction type
    fig, axes = plt.subplots(len(classes), 1, figsize=(10, 4 * len(classes)), sharex=True)

    if len(classes) == 1:
        axes = [axes]  # Ensure iterable

    for ax, itype in zip(axes, classes):
        subset = class_df[class_df['Interaction'] == itype]

        recalls = subset.set_index('Model')['Recall']
        supports = subset.set_index('Model')['Support']

        # Plot recall bars
        recalls.plot(kind='bar', ax=ax, color='purple', edgecolor='black')

        # Annotate bars with "TP / Support"
        for i, model in enumerate(recalls.index):
            tp = int(round(recalls[model] * supports[model]))
            ax.text(i, recalls[model] + 0.02, f"{tp}/{supports[model]}", 
                    ha='center', fontsize=9, rotation=0)

        ax.set_ylim(0, 1.05)
        ax.set_title(f"Class: {itype}", fontsize=14)
        ax.set_ylabel("Recall (Correct / Total)")
        ax.grid(axis="y", linestyle="--", alpha=0.6)

    plt.suptitle("Per-Class Prediction Distribution per Model", fontsize=16, y=1.02)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    global_results, class_results = evaluate_predictions(merged_df)
    report = generate_comparison_report(global_results, class_results)
    print(report)

    # 👇 Add this line to display the plots
    plot_class_distributions(class_results)

# GPT v2 model on Test Set

In [ ]:
def transform_structure(data):
    output = {'sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

transformed = transform_structure(test_entities[0])
print(transformed)

In [ ]:
def transform_structure(data):
    output = {'Sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

updated_test_entities = []
for i in range(len(test_entities)):
    transformed = transform_structure(test_entities[i])
    updated_test_entities.append(transformed)
print(updated_test_entities[-1])
len(updated_test_entities)

In [ ]:
import os
import json
import time
from google import genai


GEMINI_API_KEY = ''
assert GEMINI_API_KEY, ""
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
client = genai.Client()

# Output holder
all_responses = []

# Prompt styles
interaction_types = """
- mechanism: Pharmacokinetic interaction (metabolism, clearance, absorption, transport). Cues: inhibits/induces, substrate, CYP/UGT/P-gp, AUC/Cmax change, plasma levels ↑/↓, bioavailability, clearance.
- effect: Pharmacodynamic clinical effect or outcome. Cues: bleeding, sedation, QT prolongation, hypotension, serotonin syndrome, reduced efficacy, toxicity, adverse events.
- advice: Explicit recommendation/warning/contraindication. Cues: avoid, do not use, contraindicated, use with caution, monitor, reduce/increase dose, separate by X hours.
- int: A generic mention that an interaction exists without mechanism, effect, or advice. Cues: “interacts”, “interaction exists”, “DDI reported”, “potential interaction” with no further details.
- no interaction: Explicit statement of no interaction or safe coadministration (e.g., “no significant interaction”, “can be coadministered”, “safe with”).
"""

rules = """ Rules (strict):
A. Allowed labels (lowercase) are exactly: mechanism, effect, advice, int, no interaction.
   - Never output "advise". If your reasoning suggests "advise", output the label "advice".
B. One triplet per unique unordered pair of entities. Do not output both (Drug1, x, Drug2) and (Drug2, x, Drug1).
C. Decision hierarchy (to avoid misuse of "int"):
   1) If any recommendation/warning/contraindication language appears → advice.
   2) Else if PK level/process is stated (inhibit/induce/CYP/levels/AUC/clearance/absorption/transport) → mechanism.
   3) Else if clinical effect/outcome/AE/risk is stated (bleeding, sedation, etc.) → effect.
   4) Else if the text only states there is an interaction with no details → int.
   5) Else → no interaction.
D. “Monitor”, “use with caution”, “dose adjust”, “separate dosing” are **advice**, not int.
E. “Levels/concentrations/AUC/Cmax change”, “CYP/UGT/P-gp”, “inhibits/induces/substrate” are **mechanism**, not int.
F. “Increased risk of [event]” or any clinical outcome is **effect**, not int.
G. Output only triplets in the form: (Drug1, label, Drug2). No extra text. """

few_shot_rules = """ Rules (strict):
A. Allowed labels (lowercase) are exactly: mechanism, effect, advice, int, no interaction.
   - Never output "advise". If your reasoning suggests "advise", output the label "advice".
B. One triplet per unique unordered pair of entities. Do not output both (Drug1, x, Drug2) and (Drug2, x, Drug1).
C. Decision hierarchy (to avoid misuse of "int"):
   1) advice > 2) mechanism > 3) effect > 4) int > 5) no interaction.
D. “Monitor”, “use with caution”, “dose adjust”, “separate dosing” → advice.
E. PK cues (CYP/UGT/P-gp, levels/AUC/Cmax, inhibit/induce/substrate) → mechanism.
F. Clinical outcome/risk terms → effect.
G. Output only triplets in the exact form: (Drug1, label, Drug2). """

few_shot_examples = f"""
{interaction_types}
"""

def extract_entities(entity_dict):
    return [v for k, v in entity_dict.items() if k.startswith("Entity_")]

# Inference loop
for i in range(0, len(updated_test_entities)):
    example = updated_test_entities[i]
    sentence = example["Sentence"]
    entities = extract_entities(example)

    output_entry = {
        "index": i,
        "sentence": sentence,
        "entities": entities
    }

    # Prepare prompts
    zero_shot_prompt = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from the following text.

Definitions of interaction types: {interaction_types}
Rules are given: {rules}

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""

    few_shot_prompt_2 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

 Definitions of interaction types: {interaction_types}
 Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_2_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""
    
    few_shot_prompt_5 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types:{interaction_types}
Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_5_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""


    few_shot_prompt_10 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types: {interaction_types}
Rules are given: {few_shot_rules}

Examples: {few_shot_prompt_10_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""
    
    few_shot_prompt_20 = f"""You are a biomedical expert. Extract drug-drug interactions (DDIs) from biomedical text.

Interaction types: {interaction_types}
Rules are given: {few_shot_rules}


Examples: {few_shot_prompt_20_examples}

Now extract triplets for:

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:"""


    cot_prompt = f"""You are a biomedical expert. Perform the following:

1) List unique unordered entity pairs.
2) For each pair, locate evidence span(s) in the sentence.
3) Apply the decision hierarchy strictly (advice > mechanism > effect > int > no interaction) considering definition of labels: {interaction_types}.
4) Canonicalize the label set to: mechanism, effect, advice, int, no interaction.
   - If you would output "advise", output "advice" instead.
5) Produce exactly one triplet per pair.

Rules are given: {rules}

Text: "{sentence}"
Entities: {", ".join(entities)}

Triplets:
"""

    prompts = {
        "zero_shot_output": zero_shot_prompt,
        "few_shot_output_2": few_shot_prompt_2,
        "few_shot_output_5": few_shot_prompt_5,
        "few_shot_output_10": few_shot_prompt_10,
        "few_shot_output_20": few_shot_prompt_20,
        "cot_output": cot_prompt
    }

    for method_name, prompt in prompts.items():
        try:
            response = client.models.generate_content(
                model="gemma-3-27b-it",
                contents=prompt,
                config=genai.types.GenerateContentConfig(
                    temperature=0.0
                )
            )
            time.sleep(6)

            output_entry[method_name] = response.text.strip()
            print(f"[{method_name}] Finished index {i}")
            print(f"{method_name}: {response.text.strip()}")

        except Exception as e:
            print(f"Error at index {i} with method {method_name}: {e}")
            output_entry[method_name] = "ERROR"

    all_responses.append(output_entry)

    # Save after each example (optional safety)
    with open("gpt_method_2_predictions_on_test_set.json", "w") as f:
        json.dump(all_responses, f, indent=2)

# Evaluation on Test Set

In [ ]:
with open("gpt_method_2_predictions_on_test_set.json", "r") as f:
    all_responses = json.load(f)
all_responses[0]

In [ ]:
test_triplets_df = pd.DataFrame(test_triplets)
test_triplets_df

In [ ]:
triplet_prediction_results = pd.DataFrame(all_responses)
triplet_prediction_results

In [ ]:
merged_df = test_triplets_df.merge(triplet_prediction_results)
merged_df

In [ ]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Configuration
INTERACTION_TYPES = {"effect", "mechanism", "advice", "int", "no interaction"}
MODEL_MAP = {
    'zero_shot_output': 'Zero-Shot',
    'few_shot_output_2': 'Few-Shot-2',
    'few_shot_output_5': 'Few-Shot-5',
    'few_shot_output_10': 'Few-Shot-10',
    'few_shot_output_20': 'Few-Shot-20',
    'cot_output': 'CoT'
}
MODEL_ORDER = list(MODEL_MAP.keys())

# --- Normalization ---
def normalize_interaction_type(itype: str) -> str:
    """Map variations of interaction types to canonical form"""
    itype = itype.strip().lower()
    if itype in {"advise", "adviced", "advised", "advice"}:
        return "advice"
    return itype

def normalize_drug_pair(drug1, drug2):
    """Create canonical drug pair representation"""
    return tuple(sorted([drug1.strip().lower(), drug2.strip().lower()]))

# --- Parsing ---
def parse_prediction_output(pred_output):
    """Robust parser that requires exactly 2 drugs and 1 interaction type"""
    if not isinstance(pred_output, str):
        return []
    
    triplet_pattern = r'\(([^,]+?),\s*([^,]+?),\s*([^)]+?)\)'
    raw_triplets = re.findall(triplet_pattern, pred_output)
    
    cleaned_triplets = []
    for triplet in raw_triplets:
        parts = [p.strip().lower() for p in triplet]
        norm_parts = [normalize_interaction_type(p) for p in parts]
        
        # Identify interaction type candidates
        interaction_candidates = [p for p in norm_parts if p in INTERACTION_TYPES]
        drug_candidates = [parts[i] for i, p in enumerate(norm_parts) if p not in INTERACTION_TYPES]
        
        if len(interaction_candidates) == 1 and len(drug_candidates) == 2:
            itype = interaction_candidates[0]
            cleaned_triplets.append((drug_candidates[0], itype, drug_candidates[1]))
            continue
        
        # Fallback: positional checks
        if len(parts) == 3:
            if norm_parts[1] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], norm_parts[1], parts[2]))
            elif norm_parts[0] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[1], norm_parts[0], parts[2]))
            elif norm_parts[2] in INTERACTION_TYPES:
                cleaned_triplets.append((parts[0], norm_parts[2], parts[1]))
    
    return cleaned_triplets


# --- Evaluation ---
def evaluate_predictions(df):
    """Comprehensive evaluation with model comparison metrics"""
    results = []
    per_class_results = []
    
    for model_col in MODEL_ORDER:
        if model_col not in df.columns:
            continue
            
        model_name = MODEL_MAP[model_col]
        
        # Initialize metrics
        metrics = {
            'Model': model_name,
            'Accuracy': 0,
            'Micro_Precision': 0, 'Micro_Recall': 0, 'Micro_F1': 0,
            'Macro_Precision': 0, 'Macro_Recall': 0, 'Macro_F1': 0,
            'Weighted_Precision': 0, 'Weighted_Recall': 0, 'Weighted_F1': 0,
        }
        
        class_metrics = {itype: {'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0} for itype in INTERACTION_TYPES}
        all_true = []
        all_pred = []
        
        for _, row in df.iterrows():
            # Process ground truth
            gold_triplets = set()
            for triplet in row['triplets']:
                if len(triplet) == 3:
                    d1, itype, d2 = [x.strip().lower() for x in triplet]
                    itype = normalize_interaction_type(itype)
                    gold_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Process predictions
            pred_triplets = set()
            for d1, itype, d2 in parse_prediction_output(row[model_col]):
                itype = normalize_interaction_type(itype)
                pred_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Create lists for sklearn metrics
            for itype in INTERACTION_TYPES:
                # True positive: both gold and prediction have this interaction
                if any((dp, t) in gold_triplets and t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['TP'] += 1
                    all_true.append(itype)
                    all_pred.append(itype)
                # False positive: prediction has this interaction but gold doesn't
                elif any(t == itype for dp, t in pred_triplets):
                    class_metrics[itype]['FP'] += 1
                    all_pred.append(itype)
                    # Assign the true label
                    true_label = "no interaction"
                    for dp, t in gold_triplets:
                        if dp in [dp_p for dp_p, t_p in pred_triplets if t_p == itype]:
                            true_label = t
                            break
                    all_true.append(true_label)
                # False negative: gold has this interaction but prediction doesn't
                elif any(t == itype for dp, t in gold_triplets):
                    class_metrics[itype]['FN'] += 1
                    all_true.append(itype)
                    all_pred.append("no interaction")  # Missed prediction
        
        # Calculate accuracy
        correct = sum(1 for t, p in zip(all_true, all_pred) if t == p)
        total = len(all_true) if all_true else 1
        metrics['Accuracy'] = correct / total if total > 0 else 0
        
        # Calculate micro, macro, and weighted metrics using sklearn
        if all_true and all_pred:
            micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='micro', zero_division=0)
            macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='macro', zero_division=0)
            weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
                all_true, all_pred, average='weighted', zero_division=0)
            
            metrics.update({
                'Micro_Precision': micro_precision,
                'Micro_Recall': micro_recall,
                'Micro_F1': micro_f1,
                'Macro_Precision': macro_precision,
                'Macro_Recall': macro_recall,
                'Macro_F1': macro_f1,
                'Weighted_Precision': weighted_precision,
                'Weighted_Recall': weighted_recall,
                'Weighted_F1': weighted_f1
            })
        
        results.append(metrics)
        
        # Calculate per-class metrics
        for itype in INTERACTION_TYPES:
            cm = class_metrics[itype]
            p = cm['TP'] / (cm['TP'] + cm['FP']) if cm['TP'] + cm['FP'] > 0 else 0
            r = cm['TP'] / (cm['TP'] + cm['FN']) if cm['TP'] + cm['FN'] > 0 else 0
            f = 2 * p * r / (p + r) if p + r > 0 else 0
            support = cm['TP'] + cm['FN']
            
            per_class_results.append({
                'Model': model_name,
                'Interaction': itype,
                'Precision': p,
                'Recall': r,
                'F1': f,
                'Support': support
            })
    
    return pd.DataFrame(results), pd.DataFrame(per_class_results)

# --- Report ---
def generate_comparison_report(global_df, class_df):
    """Create model comparison report with sorting"""
    global_df = global_df.sort_values('Micro_F1', ascending=False)
    
    # Format numbers
    global_df = global_df.round(4)
    class_df = class_df.round(4)
    
    report = "MODEL COMPARISON REPORT\n"
    report += "=======================\n\n"
    report += "Global Metrics:\n"
    report += global_df.to_string(index=False) + "\n\n"
    report += "Per-Class Metrics:\n"
    report += class_df.to_string(index=False)
    
    best_model = global_df.iloc[0]
    report += f"\n\nBest Model: {best_model['Model']} (Macro F1={best_model['Macro_F1']:.4f}, Micro F1={best_model['Micro_F1']:.4f})"
    
    return report

# --- Main execution ---
if __name__ == "__main__":
    global_results, class_results = evaluate_predictions(merged_df)
    report = generate_comparison_report(global_results, class_results)
    print(report)

In [ ]:
import matplotlib.pyplot as plt

def plot_class_distributions(class_df):
    """
    Visualize correct prediction distribution per class across models.
    Each bar shows recall (correct / total).
    """
    classes = sorted(class_df['Interaction'].unique())
    models = class_df['Model'].unique()

    # Make one subplot per interaction type
    fig, axes = plt.subplots(len(classes), 1, figsize=(10, 4 * len(classes)), sharex=True)

    if len(classes) == 1:
        axes = [axes]  # Ensure iterable

    for ax, itype in zip(axes, classes):
        subset = class_df[class_df['Interaction'] == itype]

        recalls = subset.set_index('Model')['Recall']
        supports = subset.set_index('Model')['Support']

        # Plot recall bars
        recalls.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

        # Annotate bars with "TP / Support"
        for i, model in enumerate(recalls.index):
            tp = int(round(recalls[model] * supports[model]))
            ax.text(i, recalls[model] + 0.02, f"{tp}/{supports[model]}", 
                    ha='center', fontsize=9, rotation=0)

        ax.set_ylim(0, 1.05)
        ax.set_title(f"Class: {itype}", fontsize=14)
        ax.set_ylabel("Recall (Correct / Total)")
        ax.grid(axis="y", linestyle="--", alpha=0.6)

    plt.suptitle("Per-Class Prediction Distribution per Model", fontsize=16, y=1.02)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    global_results, class_results = evaluate_predictions(merged_df)
    report = generate_comparison_report(global_results, class_results)
    print(report)

    # 👇 Add this line to display the plots
    plot_class_distributions(class_results)

In [ ]:
import re

# Define valid interaction types
INTERACTION_TYPES = {"effect", "mechanism", "advice", "int", "no interaction"}

def process_triplets(df):
    # Step 1: Explode ground truth triplets
    exploded_rows = []
    for _, row in merged_df.iterrows():
        for triplet in row['triplets']:
            exploded_rows.append({
                'index': row['index'],
                'sentence': row['sentence'],
                'ground_truth_triplet': triplet,
                'ground_truth_type': triplet[1],  # interaction_type from triplet
                'zero_shot_output': row['zero_shot_output'],
                'few_shot_output_2': row['few_shot_output_2'],
                'few_shot_output_5': row['few_shot_output_5'],
                'few_shot_output_10': row['few_shot_output_10'],
                'few_shot_output_20': row['few_shot_output_20'],
                'cot_output': row['cot_output'],
            })
    df_expanded = pd.DataFrame(exploded_rows)
    return df_expanded

def parse_prediction_output(pred_output):
    """Parse prediction output into cleaned triplets with consistent format"""
    if not isinstance(pred_output, str):
        return []
    
    # Find all triplets in the string
    triplet_pattern = r'\(([^,]+?),\s*([^,]+?),\s*([^)]+?)\)'
    raw_triplets = re.findall(triplet_pattern, pred_output)
    
    cleaned_triplets = []
    for triplet in raw_triplets:
        parts = [p.strip() for p in triplet]
        
        # Case 1: Middle element is interaction type
        if parts[1].lower() in INTERACTION_TYPES:
            cleaned_triplets.append((parts[0], parts[1].lower(), parts[2]))
        
        # Case 2: Last element is interaction type
        elif parts[2].lower() in INTERACTION_TYPES:
            cleaned_triplets.append((parts[0], parts[2].lower(), parts[1]))
        
        # Case 3: Try to identify interaction by position
        else:
            # Try first element as interaction
            if parts[0].lower() in INTERACTION_TYPES:
                cleaned_triplets.append((parts[1], parts[0].lower(), parts[2]))
            # Try last element as interaction (even if not in set)
            elif parts[2].lower() not in INTERACTION_TYPES and parts[1].lower() not in INTERACTION_TYPES:
                # Final fallback: assume last element is interaction
                cleaned_triplets.append((parts[0], parts[2].lower(), parts[1]))
    
    return cleaned_triplets

def match_predictions(df_expanded, column_name):
    """Match predictions to ground truth with per-type tracking"""
    matched_data = []
    
    for _, row in df_expanded.iterrows():
        # Get ground truth triplet
        gt_drug1, gt_interaction, gt_drug2 = row['ground_truth_triplet']
        gt_interaction = gt_interaction.lower()
        gt_drug_pair = tuple(sorted([gt_drug1.strip(), gt_drug2.strip()]))
        
        # Parse predictions
        predictions = parse_prediction_output(row[column_name])
        pred_matches = {tuple(sorted([d1.strip(), d2.strip()])): itype 
                        for d1, itype, d2 in predictions}
        
        # Check match for this specific triplet
        match_found = False
        if gt_drug_pair in pred_matches:
            match_found = (pred_matches[gt_drug_pair] == gt_interaction)
        
        matched_data.append({
            'index': row['index'],
            'sentence': row['sentence'],
            'ground_truth_triplet': (gt_drug1, gt_interaction, gt_drug2),
            'ground_truth_type': gt_interaction,
            'predicted_triplets': predictions,
            'match': match_found,
            'drug_pair': gt_drug_pair
        })
    
    return pd.DataFrame(matched_data)

# Generate matched dataframes
df_expanded = process_triplets(merged_df)  # Ensure this creates one row per triplet

matched_dfs = {
    'zero_shot': match_predictions(df_expanded, 'zero_shot_output'),
    'few_shot_2': match_predictions(df_expanded, 'few_shot_output_2'),
    'few_shot_5': match_predictions(df_expanded, 'few_shot_output_5'),
    'few_shot_10': match_predictions(df_expanded, 'few_shot_output_10'),
    'few_shot_20': match_predictions(df_expanded, 'few_shot_output_20'),
    'cot': match_predictions(df_expanded, 'cot_output')
}

# Calculate per-type accuracy
results = []

for method_name, df in matched_dfs.items():
    # Overall accuracy
    overall_acc = df['match'].mean()
    
    # Per-type accuracy
    type_acc = {}
    for itype in INTERACTION_TYPES:
        type_df = df[df['ground_truth_type'] == itype]
        if len(type_df) > 0:
            type_acc[itype] = type_df['match'].mean()
        else:
            type_acc[itype] = None  # Handle missing types
    
    results.append({
        'method': method_name,
        'overall_accuracy': overall_acc,
        **{f'{itype}_accuracy': acc for itype, acc in type_acc.items()}
    })

# Create results dataframe
results_df = pd.DataFrame(results)

In [ ]:
def calculate_pair_and_interaction_counts(merged_df):
    TECHNIQUE_ORDER = [
        'zero_shot_output',
        'few_shot_output_2',
        'few_shot_output_5',
        'few_shot_output_10',
        'few_shot_output_20',
        'cot_output'
    ]

    TECHNIQUE_MAP = {
        'zero_shot_output': 'Zero-Shot',
        'few_shot_output_2': 'Few-Shot (2)',
        'few_shot_output_5': 'Few-Shot (5)',
        'few_shot_output_10': 'Few-Shot (10)',
        'few_shot_output_20': 'Few-Shot (20)',
        'cot_output': 'Chain-of-Thought'
    }
    
    per_method_details = []
    per_interaction_details = []
    
    for tech_col in TECHNIQUE_ORDER:
        if tech_col not in merged_df.columns:
            continue
        
        tech_name = TECHNIQUE_MAP.get(tech_col, tech_col)
        
        interaction_correct, interaction_total = 0, 0
        per_class = {itype: {'tp': 0, 'fn': 0} for itype in INTERACTION_TYPES}
        
        for _, row in merged_df.iterrows():
            # Ground truth triplets
            gold_triplets = set()
            if isinstance(row['triplets'], list):
                for triplet in row['triplets']:
                    if len(triplet) == 3:
                        d1, itype, d2 = [x.strip().lower() for x in triplet]
                        itype = normalize_interaction_type(itype)  # <-- FIX
                        gold_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Predictions
            pred_triplets = set()
            for d1, itype, d2 in parse_prediction_output(row[tech_col]):
                itype = normalize_interaction_type(itype)  # <-- FIX
                pred_triplets.add((normalize_drug_pair(d1, d2), itype))
            
            # Pair-level counts (correct if both pair+interaction match)
            gold_pairs = {dp for dp, _ in gold_triplets}
            for dp, it in pred_triplets:
                if dp in gold_pairs:  # model predicted some interaction for a true pair
                    interaction_total += 1
                    if (dp, it) in gold_triplets:
                        interaction_correct += 1
            
            # Per-interaction type counts
            for itype in INTERACTION_TYPES:
                gold_sub = {(dp, it) for dp, it in gold_triplets if it == itype}
                pred_sub = {(dp, it) for dp, it in pred_triplets if it == itype}
                
                tp = len(gold_sub & pred_sub)
                fn = len(gold_sub - pred_sub)
                
                per_class[itype]['tp'] += tp
                per_class[itype]['fn'] += fn
        
        # Store per-method totals
        per_method_details.append({
            "Technique": tech_name,
            "Correct Pairs": interaction_correct,
            "Total Pairs": interaction_total
        })
        
        # Store per-interaction type totals
        for itype in INTERACTION_TYPES:
            tp = per_class[itype]['tp']
            fn = per_class[itype]['fn']
            per_interaction_details.append({
                "Technique": tech_name,
                "Interaction Type": itype,
                "Correct": tp,
                "Total": tp + fn
            })
    
    return pd.DataFrame(per_method_details), pd.DataFrame(per_interaction_details)
method_df, interaction_df = calculate_pair_and_interaction_counts(merged_df)
print("PER-METHOD PAIR COUNTS:")
print(method_df.to_string(index=False))
print("\nPER-INTERACTION TYPE COUNTS:")
print(interaction_df.to_string(index=False))